In [2]:
#default_exp dag.buoy_data

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
#export
from huey.imports import *
from huey.dag.airflow_imports import *

[2020-07-06 07:06:50,899] {settings.py:254} INFO - settings.configure_orm(): Using pool settings. pool_size=5, max_overflow=10, pool_recycle=1800, pid=12956


In [5]:
#export
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'start_date': datetime(2019, 7, 24),
    'email': ['heyhueyapp@gmail.com'],
    'email_on_failure': True,
    'email_on_retry': True,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
    # 'queue': 'bash_queue',
    # 'pool': 'backfill',
    # 'priority_weight': 10,
    # 'end_date': datetime(2016, 1, 1),
}

In [6]:
#export
# @hourly
dag = DAG(
    'huey_import_buoy_data', \
    default_args=default_args, \
    schedule_interval="0 * * * *", \
    catchup=False
)

In [7]:
dag

<DAG: huey_import_buoy_data>

# Get all active data buoys

In [8]:
from huey.data.sql import get_db_session

In [41]:
#export
from huey.data.sql import get_db_engine

In [53]:
#export
def get_all_active_buoy_station_ids():
    db_engine = get_db_engine()
    df_buoys = pd.read_sql("select * from buoy where data_download_active = 't'", db_engine)
    return df_buoys.station_id

In [12]:
#db_engine = PostgresHook(postgres_conn_id=self.db_env).get_sqlalchemy_engine(engine_kwargs=dict(poolclass=NullPool))

In [13]:
#buoys =  session.query(Buoy).filter(Buoy.data_download_active == True).all()

In [54]:
active_station_ids = get_all_active_buoy_station_ids()

In [55]:
len(active_station_ids)

95

In [56]:
active_station_ids[0]

'SMOC1'

# Import realtime wave data

In [58]:
#export
from huey.data.importer.buoy_data import import_buoy_realtime_wave_detail

In [59]:
#export
def run_import_buoy_realtime_wave_detail(station_id):
    import_buoy_realtime_wave_detail(station_id)

In [63]:
t1 = PythonOperator(
    task_id='import_buoy_realtime_wave_detail',
    python_callable=run_import_buoy_realtime_wave_detail,
    op_kwargs={'station_id': station_id},
    dag=dag)

In [61]:
t1.execute(context={})

Slack is broken in 1.10.7, fix is in 1.10.8.
FIXME: upgrade to 1.10.8 when its out.

In [64]:
!airflow list_dags

[2020-07-06 07:22:29,928] {settings.py:254} INFO - settings.configure_orm(): Using pool settings. pool_size=5, max_overflow=10, pool_recycle=1800, pid=13462
[2020-07-06 07:22:30,613] {__init__.py:51} INFO - Using executor SequentialExecutor
[2020-07-06 07:22:30,613] {dagbag.py:403} INFO - Filling up the DagBag from /Users/yipstar/airflow/dags
[2020-07-06 07:22:31,354] {dagbag.py:246} ERROR - Failed to import: /Users/yipstar/airflow/dags/huey_dags/buoy_data_dag.py
Traceback (most recent call last):
  File "/Users/yipstar/anaconda/envs/prophet/lib/python3.7/site-packages/airflow/models/dagbag.py", line 243, in process_file
    m = imp.load_source(mod_name, filepath)
  File "/Users/yipstar/anaconda/envs/prophet/lib/python3.7/imp.py", line 171, in load_source
    module = _load(spec)
  File "<frozen importlib._bootstrap>", line 696, in _load
  File "<frozen importlib._bootstrap>", line 677, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "

In [23]:
!airflow list_tasks huey_import_buoy_data

[2020-07-06 07:08:51,137] {settings.py:254} INFO - settings.configure_orm(): Using pool settings. pool_size=5, max_overflow=10, pool_recycle=1800, pid=13023
[2020-07-06 07:08:51,770] {__init__.py:51} INFO - Using executor SequentialExecutor
[2020-07-06 07:08:51,770] {dagbag.py:403} INFO - Filling up the DagBag from /Users/yipstar/airflow/dags
import_buoy_raw_spectral_wave_data
import_buoy_realtime_wave_detail
send_slack_success_alert


In [34]:
#for i, station_id in df_buoys.station_id.iteritems():
    #print(station_id)

## All Active Buoys

In [78]:
#export

import_buoy_realtime_wave_detail_tasks = []
active_station_ids = get_all_active_buoy_station_ids()

for station_id in active_station_ids:
    
    task_id = f"import_buoy_realtime_wave_detail-{station_id}"

    task = PythonOperator(task_id=task_id,
                          python_callable=run_import_buoy_realtime_wave_detail,
                          op_kwargs={'station_id': station_id},
                          dag=dag)
    
    import_buoy_realtime_wave_detail_tasks.append(task)

[2020-07-06 07:31:53,018] {base_hook.py:84} INFO - Using connection to: id: huey_dev. Host: localhost, Port: 54320, Schema: huey_dev, Login: postgres, Password: XXXXXXXX, extra: None


In [37]:
import_buoy_realtime_wave_detail_tasks;

In [38]:
!airflow list_tasks huey_import_buoy_data

[2020-07-06 07:16:59,494] {settings.py:254} INFO - settings.configure_orm(): Using pool settings. pool_size=5, max_overflow=10, pool_recycle=1800, pid=13281
[2020-07-06 07:17:00,487] {__init__.py:51} INFO - Using executor SequentialExecutor
[2020-07-06 07:17:00,487] {dagbag.py:403} INFO - Filling up the DagBag from /Users/yipstar/airflow/dags
import_buoy_raw_spectral_wave_data
import_buoy_realtime_wave_detail
send_slack_success_alert


# import_buoy_raw_spectral_wave_data

In [ ]:
#export
from huey.data.importer.buoy_data import import_buoy_raw_spectral_wave_data

def run_import_buoy_raw_spectral_wave_data():
    station_id = "46025"
    import_buoy_raw_spectral_wave_data(station_id)

In [ ]:
data = run_import_buoy_raw_spectral_wave_data()

[2020-01-22 13:33:14,079] {base_hook.py:84} INFO - Using connection to: id: huey_dev. Host: localhost, Port: None, Schema: huey_dev, Login: postgres, Password: XXXXXXXX, extra: None
observation for date: 2020-01-22 21:00:00+00:00 already present, skipping.
import complete


In [ ]:
#export
t2 = PythonOperator(
    task_id='import_buoy_raw_spectral_wave_data',
    python_callable=run_import_buoy_raw_spectral_wave_data,
    dag=dag)

In [ ]:
t2.execute(context={})

observation for date: 2020-01-22 21:00:00+00:00 already present, skipping.
import complete


# Slack Alert

In [73]:
#export
slack_token = BaseHook.get_connection('slack').password
slack_success_task = SlackAPIPostOperator(
    task_id='send_slack_success_alert',
    token=slack_token,
    text='Buoy Data Import Success.',
    channel='#huey_data_import',
    username='heyhueyapp',
    dag=dag,
    blocks=[],
    attachments=[]
)

In [ ]:
t3.execute(context={})

In [74]:
#export

import_buoy_realtime_wave_detail_tasks >> slack_success_task

<Task(SlackAPIPostOperator): send_slack_success_alert>

# Export

In [79]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_data.sql.ipynb.
Converted 02_data.importer.buoy_data.ipynb.
Converted 03_dag.buoy_data.ipynb.
Converted 04_data.models.buoy.ipynb.
Converted 05_data.models.log_entry.ipynb.
Converted 06_data.exporter.ipynb.


In [ ]:
from huey.dag.buoy_data import run_import_buoy_realtime_wave_detail
run_import_buoy_realtime_wave_detail()

In [76]:
!airflow list_dags

/Users/yipstar/anaconda/envs/prophet/lib/python3.7/site-packages/sqlalchemy/ext/declarative/clsregistry.py:129: SAWarning: This declarative base already contains a class with the same class name and module name as huey.data.models.buoy.Buoy, and will be replaced in the string-lookup table.
  % (item.__module__, item.__name__)
/Users/yipstar/anaconda/envs/prophet/lib/python3.7/site-packages/sqlalchemy/ext/declarative/clsregistry.py:129: SAWarning: This declarative base already contains a class with the same class name and module name as huey.data.models.buoy.BuoyRawSpectralWaveData, and will be replaced in the string-lookup table.
  % (item.__module__, item.__name__)
[autoreload of huey.data.models.buoy failed: Traceback (most recent call last):
  File "/Users/yipstar/anaconda/envs/prophet/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/yipstar/anaconda/envs/prophet/lib/python3.7/site-packages/

[2020-07-06 07:29:04,030] {settings.py:254} INFO - settings.configure_orm(): Using pool settings. pool_size=5, max_overflow=10, pool_recycle=1800, pid=13741
[2020-07-06 07:29:04,733] {__init__.py:51} INFO - Using executor SequentialExecutor
[2020-07-06 07:29:04,734] {dagbag.py:403} INFO - Filling up the DagBag from /Users/yipstar/airflow/dags
[2020-07-06 07:29:05,450] {base_hook.py:84} INFO - Using connection to: id: huey_dev. Host: localhost, Port: 54320, Schema: huey_dev, Login: postgres, Password: XXXXXXXX, extra: None


-------------------------------------------------------------------
DAGS
-------------------------------------------------------------------
huey_import_buoy_data



In [77]:
!airflow list_tasks huey_import_buoy_data

[2020-07-06 07:29:06,404] {settings.py:254} INFO - settings.configure_orm(): Using pool settings. pool_size=5, max_overflow=10, pool_recycle=1800, pid=13749
[2020-07-06 07:29:06,934] {__init__.py:51} INFO - Using executor SequentialExecutor
[2020-07-06 07:29:06,934] {dagbag.py:403} INFO - Filling up the DagBag from /Users/yipstar/airflow/dags
[2020-07-06 07:29:07,525] {base_hook.py:84} INFO - Using connection to: id: huey_dev. Host: localhost, Port: 54320, Schema: huey_dev, Login: postgres, Password: XXXXXXXX, extra: None
import_buoy_raw_spectral_wave_data
import_buoy_raw_spectral_wave_data-46011
import_buoy_raw_spectral_wave_data-46023
import_buoy_raw_spectral_wave_data-46025
import_buoy_raw_spectral_wave_data-46028
import_buoy_raw_spectral_wave_data-46042
import_buoy_raw_spectral_wave_data-46044
import_buoy_raw_spectral_wave_data-46047
import_buoy_raw_spectral_wave_data-46053
import_buoy_raw_spectral_wave_data-46054
import_buoy_raw_spectral_wave_data-46062
import_buoy_raw_spectral_wa